#### Scraping Dynamic Pages w/ Requests ❎

In [ ]:
import requests
import pandas as pd

response = requests.get('https://www.premierleague.com/stats/top/players/goals?se=363')

df = pd.read_html(response.text)
df[0] # Loads the all time list, not the current season data (/goals?se=363)...also, how are we going to pagnate to the next page of data?

#### Scraping Dynamic Pages w/ Watir (Nerodia) & Chromium (Chrome) ✅
> https://github.com/watir/nerodia

In [ ]:
from nerodia.browser import Browser
import pandas as pd
import time

browser = Browser('chrome', headless=True) # Set Headless to True so the physical GUI of Chrome doesn't have to be used ⚡️ 
browser.goto('https://www.premierleague.com/stats/top/players/goals?se=363') # Now use the browser to navigate to the EPL Stats Page

time.sleep(5) # Allow data time to load into HTML

In [ ]:
goals_df = pd.read_html(browser.html)[0] # Use Pandas to fetch all the tables within the browser html, select the first table it finds ([0])

# Note: On the EPL site, when you've reached the end of the table, the table's Page Next element has 'inactive' added to the elements class. Use browser tools to inspect the Page Next button on the last page of the goals table to see for yourself.
# Note: As we know this, we can keep clicking the page next button and scraping the table until the button is inactive. In Python we can use while:
while ((browser.div(class_name=["paginationBtn", "paginationNextContainer", "inactive"])).exists == False):
  browser.div(class_name=["paginationBtn", "paginationNextContainer"]).fire_event('onClick') # fire onClick event on page next element. If it was a button element (not a div element), we could simply use .click() 
  time.sleep(2)
  goals_df = goals_df.append(pd.read_html(browser.html)[0]) # append the table from this page with the existing goals dataframe.
  print("Next Page")

goals_df = goals_df[goals_df['Stat'] > 0] # Random Players at end of table with 0 goals...

goals_df.to_csv(r'epl_goals.csv', index=False) # Save dataframe to new csv file

browser.close() # Close Browser